In [ ]:
!python --version

In [ ]:
!pip list

In [ ]:
!pip install transformers sentencepiece==0.1.97 fugashi ipadic datasets unidic-lite

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# row_data = pd.read_csv('/content/drive/MyDrive/ML/sample.csv')
row_data = pd.read_csv('./sample.csv')

class_le = LabelEncoder()
row_data["label"] = class_le.fit_transform(row_data['label'].values)

row_data

In [ ]:
from transformers import pipeline

classifier = pipeline("summarization", model="megagonlabs/t5-base-japanese-web", tokenizer="megagonlabs/t5-base-japanese-web", device=0)

In [ ]:
from sklearn.model_selection import train_test_split

train, valid_test = train_test_split(row_data, test_size=0.2, shuffle=True, random_state=1)
valid, test = train_test_split(valid_test, test_size=0.5, shuffle=True, random_state=1)

In [ ]:
from datasets import Dataset, DatasetDict

dataset = DatasetDict({"train":Dataset.from_pandas(train), "test": Dataset.from_pandas(test), "validation": Dataset.from_pandas(valid)})
dataset["train"].features

In [ ]:
from transformers import AutoTokenizer
import torch
from transformers import AutoModelForSequenceClassification

model_ckpt = "cl-tohoku/bert-base-japanese-v2"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_labels = len(class_le.classes_)

model = (AutoModelForSequenceClassification
    .from_pretrained(
        model_ckpt,
        num_labels=num_labels
    )
    .to(device)
)

In [ ]:
def tokenize(batch):
    return tokenizer(batch["title"], padding=True, truncation=True)

dataset_encoded = dataset.map(tokenize, batched=True, batch_size=None)

In [ ]:
dataset_encoded

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoConfig

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_labels = len(class_le.classes_)

model = (AutoModelForSequenceClassification
    .from_pretrained(
        model_ckpt,
        num_labels=num_labels
    )
    .to(device)
)

# Dropout
# model = (AutoModelForSequenceClassification
#     .from_pretrained(
#         model_ckpt,
#         config=AutoConfig.from_pretrained(
#           model_ckpt,
#           hidden_dropout_prob=0.2,
#           attention_probs_dropout_prob=0.2,
#           num_labels=num_labels
#         )
#     )
#     .to(device)
# )

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="macro")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [ ]:
from transformers import TrainingArguments

batch_size = 16
logging_steps = len(dataset_encoded["train"])
model_name = "mail_classification_model"

training_args = TrainingArguments(
    output_dir=model_name,
    num_train_epochs=2,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.0001,
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    save_strategy='epoch',
    push_to_hub=False,
    # label_smoothing_factor=0.1
)

In [ ]:
from transformers import Trainer

class CustomTrainer(Trainer):
    # focal loss
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs[0]
        alpha = 0.25
        gamma = 2
        ce_loss = torch.nn.CrossEntropyLoss(reduction='none')(logits, labels)
        pt = torch.exp(-ce_loss)
        focal_loss = alpha * (1-pt)**gamma * ce_loss
        loss = torch.mean(focal_loss)
        return (loss, outputs) if return_outputs else loss

trainer = CustomTrainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset_encoded["train"],
    eval_dataset=dataset_encoded["validation"],
    tokenizer=tokenizer
)
trainer.train()

In [ ]:
preds_output = trainer.predict(dataset_encoded["validation"])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

print(compute_metrics(preds_output))

y_preds = np.argmax(preds_output.predictions, axis=1)
y_valid = np.array(dataset_encoded["validation"]["label"])

labels = [i for i in range(len(class_le.classes_))]

In [ ]:
id2label = {}
for i in range(len(class_le.classes_)):
    id2label[i] = class_le.classes_[i]

label2id = {}
for i in range(len(class_le.classes_)):
    label2id[class_le.classes_[i]] = i

trainer.model.config.id2label = id2label
trainer.model.config.label2id = label2id

In [ ]:
from matplotlib.axis import font_manager
def plot_confusion_matrix(y_preds, y_true, labels):
    cm = confusion_matrix(y_true, y_preds)
    fig, ax = plt.subplots(figsize=(6, 6))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    disp.plot(cmap="Blues", values_format=".2f", ax=ax, colorbar=False)
    plt.title("Confusion matrix")
    plt.show()

plot_confusion_matrix(y_preds, y_valid, labels)
print(id2label)

In [ ]:
# test data
preds_output_test = trainer.predict(dataset_encoded["test"])
print(compute_metrics(preds_output_test))

y_preds_test = np.argmax(preds_output_test.predictions, axis=1)
y_test = np.array(dataset_encoded["test"]["label"])

plot_confusion_matrix(y_preds_test, y_test, labels)
print(id2label)

In [ ]:
trainer.save_model(f"/content/drive/MyDrive/ML/mail_classification_model")